In [1]:
import requests
from datetime import datetime
import pandas as pd
import re 
import csv
import time
import mysql.connector
import pymysql, os, json
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine


In [2]:
now = datetime.today()

In [3]:
list_province_in_thai = []
list_district_in_thai = []
list_subdistrict_in_thai = []
label = pd.read_excel('../code/geocodelabel.xlsx', index_col=None)
for i in range(len(label)):
    list_province_in_thai.append(label['name_th'][i])
    list_district_in_thai.append(label['district_name_th'][i])
    list_subdistrict_in_thai.append(label['subdistrict_name_th'][i])

# clear dup
list_province_in_thai = list(dict.fromkeys(list_province_in_thai))
list_district_in_thai = list(dict.fromkeys(list_district_in_thai))
list_subdistrict_in_thai = list(dict.fromkeys(list_subdistrict_in_thai))


In [4]:
def getDatafromAPI():
    url = "https://apilsm.buzzebees.com/api/place?agencyId=40839&center=13.880890218108433%2C100.56329139360224&device_locale=1033&distance=1000000&mode=nearby&require_campaign=0&within_area=0"
    headers = {'Accept': 'application/json, text/plain, */*'}
    params = {"store_format": "familymart" }
    response = requests.get(url, headers=headers , params=params)
    #response = requests.post(url, headers=headers)
    payload = response.json()
    with open(f"../trash/lawson{str(now.date())}.json"  , "w") as outfile:
        json.dump(payload, outfile)


In [5]:
getDatafromAPI()

In [6]:
def longdoAddress(lat,lng):
    url = "https://api.longdo.com/map/services/address?"
    params = {'lat': lat , 'lon' : lng , 'key' : '1917e02b4ab264ba547fecdd6e82b4aa'}
    response =requests.get(url, params=params)
    payload = response.json()
    #print("------------------------------------------")
    #print(payload)
    #print("------------------------------------------")
    province = removeWrongkeyword(payload['province'])
    district = removeWrongkeyword(payload['district'])
    subdistrict = removeWrongkeyword(payload['subdistrict'])
    try:
        postcode = payload['postcode']
    except :
        postcode = ""
    
    time.sleep(1)
    
    return province,district,subdistrict,postcode

In [7]:
def removeWrongkeyword(word):
    payload_word = word
    if(re.findall(r"\จ+[.]",word)):
        payload_word = re.sub(r"\จ+[.]","จังหวัด",word)
    elif(re.findall(r"\อ+[.]",word)):
        payload_word = re.sub(r"\อ+[.]","อำเภอ",word)
    elif(re.findall(r"\ต+[.]",word)):
        payload_word = re.sub(r"\ต+[.]","ตำบล",word)
    return payload_word
        

In [8]:
def extracteachDetail(word):
    payload_word = word
    if(re.findall(r"\จังหวัด",payload_word)):
        payload_word = re.sub("จังหวัด","",payload_word)
    elif(re.findall(r"\อำเภอ",payload_word)):
        payload_word = re.sub("อำเภอ","",payload_word)
    elif(re.findall(r"\ตำบล",payload_word)):
        payload_word = re.sub("ตำบล","",payload_word)
    elif(re.findall(r"\เขต",payload_word)):
        payload_word = re.sub("เขต","",payload_word)
    elif(re.findall(r"\แขวง",payload_word)):
        payload_word = re.sub("แขวง","",payload_word)
        
    return payload_word

In [9]:
pro = open(f'../trash/lawson{str(now.date())}.json')
payload = json.load(pro)

In [10]:
payload[0]['id']

'IC139557'

In [11]:
def getAddressDetail(address,lat,lng,store_id):
    
    str_address =  re.sub(r'["]',"",address)
    str_address = str_address.replace("  "," ")
    
    length = len(str_address)

    #print("index of: " + str(i))
    
    '''
    print(lat)
    print(lng)
    print(str_address)
    print("------------------------")
    '''
    
    #remove newline 
    if("\n" in str_address) :
        str_address = re.sub('\r','',str_address)
        str_address = re.sub('\n',' ',str_address)
    
    temp_fix = str_address.split()
    
    payload_address = ""
    
    #fix missing or wrongkeyword section 
    if(len(temp_fix) == 0):
        province,district,subdistrict,postcode = longdoAddress(lat,lng)
        payload_address = "-"
        print(temp_fix)
        print("------------------------")
    else :
        
        if(not re.findall("[0-9]", temp_fix[-1])) :
            province,district,subdistrict,postcode = longdoAddress(lat,lng)
            temp_fix.append(postcode)
        
        
        if(re.findall("[0-9]", temp_fix[-1]) and re.findall("[ก-ฮ]", temp_fix[-1])) :
                province,district,subdistrict,postcode = longdoAddress(lat,lng)
                temp_fix.remove(temp_fix[-1])
                temp_fix.append(province)
                temp_fix.append(postcode)
                temp_fix[-3] = district
                temp_fix[-4] = subdistrict
                #print(temp_fix)
        
        
        
        if re.findall("กทม.", temp_fix[-2] ) or re.findall("กรุงเทพฯ", temp_fix[-2]) or  re.findall("กรุงเทพ", temp_fix[-2]):
             temp_fix[-2] = "จังหวัดกรุงเทพมหานคร"
                
        temp_fix[-2] = removeWrongkeyword(temp_fix[-2])
        temp_fix[-3] = removeWrongkeyword(temp_fix[-3])
        temp_fix[-4] = removeWrongkeyword(temp_fix[-4])
        
        if(not re.findall(r"\จังหวัด", temp_fix[-2])):
            temp_fix[-2] = "จังหวัด" +  temp_fix[-2]
            
        postcode = extracteachDetail(temp_fix[-1])
        province = extracteachDetail(temp_fix[-2])
        district = extracteachDetail(temp_fix[-3])
        subdistrict = extracteachDetail(temp_fix[-4])
        
        for add in temp_fix :
            payload_address = payload_address + add + " "
        
        if(store_id == "IC126291"):
            payload_address = "1250-1252 (Dawnstairs) , ซอยสุขเจริญ แขวงคลองเตย เขตคลองเตย จังหวัดกรุงเทพมหานคร 10110"
            postcode = "10110"
            province = "กรุงเทพมหานคร"
            district = "คลองเตย"
            subdistrict = "คลองเตย"
        
        if(store_id == "IC126244"):
            payload_address = "169 ถนนลงหาดบางแสน ตำบลแสนสุข อำเภอเมืองชลบุรี จังหวัดชลบุรี 20130"
            postcode = "20130"
            province = "ชลบุรี"
            district = "เมืองชลบุรี"
            subdistrict = "แสนสุข"
        
    is_find = False
    
    if(province == "กรุงเทพฯ"):
        province = province.replace("กรุงเทพฯ","กรุงเทพมหานคร")
    if(province not in list_province_in_thai):
        print(str_address)
        is_find = True
        province,district,subdistrict,postcode = longdoAddress(lat,lng)
        postcode = extracteachDetail(postcode)
        province = extracteachDetail(province)
        district = extracteachDetail(district)
        subdistrict = extracteachDetail(subdistrict)
    if(district not in list_district_in_thai):
        if(not is_find):
            print(str_address)
            is_find = True
            province,district,subdistrict,postcode = longdoAddress(lat,lng)
            postcode = extracteachDetail(postcode)
            province = extracteachDetail(province)
            district = extracteachDetail(district)
            subdistrict = extracteachDetail(subdistrict)
    if(subdistrict not in subdistrict):
        if(not is_find):
            print(str_address)
            is_find = True
            province,district,subdistrict,postcode = longdoAddress(lat,lng)
            postcode = extracteachDetail(postcode)
            province = extracteachDetail(province)
            district = extracteachDetail(district)
            subdistrict = extracteachDetail(subdistrict)
        
        
        print("index of: " + str(i))
        print("address: " + payload_address)
        print("province: " + province)
        print("district: " + district)
        print("subdistrict: " + subdistrict)
        print("postcode: " + postcode)
        
        
        print("------------------------")
        
    return payload_address,province,district,subdistrict,postcode

In [12]:
length = len(payload)
i = 0

list_payload = []
while i < length :
    
    if i == 24 :
        i = i + 1
    name = payload[i]['name']
    store_id = payload[i]['id']
    lat = payload[i]['location']['latitude']
    lng = payload[i]['location']['longitude']
    address = payload[i]['address']
    address_en = payload[i]['address_en']
    
    payload_address,province,district,subdistrict,postcode = getAddressDetail(address,lat,lng,store_id)
    
    store_type = "Convenience Stores"
    store_type_name = "Lawson 108"
    
    dict_payload = { "name": name,
                    "store_id" : store_id,
                    "name_store" : store_type_name, 
                    "store_type" : store_type ,
                    "address" : payload_address ,
                    "lat" : lat,
                    "lng" : lng,
                    "subdistrict": subdistrict ,
                    "district": district,
                    "province": province}
    
    list_payload.append(dict_payload)
    
    i = i + 1


with open(f"../trash/Lawson108{str(now.date())}.json" , "w") as outfile:
        json.dump(list_payload, outfile)
    

ยูนิตที่ 21-23  สถานีรถไฟฟ้าสถานีพหลโยธิน  กรุงเทพมหานคร
[]
------------------------

[]
------------------------

[]
------------------------

[]
------------------------



In [13]:
length

111

In [14]:
df_json =  pd.read_json(f"../trash/Lawson108{str(now.date())}.json")
df_json.to_excel(f"../excel/Lawson108/Lawson108{str(now.date())}.xlsx")
df_json.to_csv(f"../csv/Lawson108/Lawson108{str(now.date())}.csv")

In [ ]:
#lawson108
ls = pd.read_csv(f'../csv/Lawson108/Lawson108{str(now.date())}.csv', header = 0)
engine = create_engine('mysql://root:@localhost:3306/lawson108')
with engine.connect() as conn, conn.begin():
    ls.to_sql(f'Lawson108{str(now.date())}', conn, if_exists='append', index=False)
conn.close()